# Adding parameters used in the synapsis physiology recipes

## Context


See ticket [BBPP134-233](https://bbpteam.epfl.ch/project/issues/secure/RapidBoard.jspa?rapidView=340&view=detail&selectedIssue=BBPP134-233)

## Imports

In [1]:
import os
import json
import re
import pandas as pd
import copy
import rdflib
import yaml
import pandas as pd
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode, SKOS
from rdflib.paths import OneOrMore

from bmo.ontologies import subontology_from_term
from bmo.utils import BMO, BRAIN_REGION_ONTOLOGY_URI, NSG, SCHEMAORG, SHACL, NXV

Read YAML files

In [2]:
with open('../data/synapse_types/E1_PT.yaml') as f:
    # use safe_load instead load
    e1_yaml = yaml.safe_load(f)

In [3]:
with open('../data/synapse_types/ProbAMPANMDA_EMS.yaml') as f:
    # use safe_load instead load
    probAMPA_yaml = yaml.safe_load(f)

In [4]:
with open('../data/synapse_types/ProbGABAAB_EMS.yaml') as f:
    # use safe_load instead load
    probGABA_yaml = yaml.safe_load(f)

### Functional parameters information

In [5]:
definitions= {
    "gsyn": "the peak conductance (in nS) for a single synaptic contact, following a Gamma distribution",
    "d": "time constant (in ms) for recovery from depression, following a Gamma distribution",
    "f": "time constant (in ms) for recovery from facilitation, following a Gamma distribution",
    "u": "utilization of synaptic efficacy, following a truncated Normal distribution",
    "dtc": "decay time constant (in ms), following a truncated Normal distribution",
    "nrrp": "number of vesicles in readily releasable pool, following a Poisson distribution",
    "gsynSRSF": "the scale factor for the conductance; SRSF: 'synaptic receptor scaling factor'",
    "uHillCoefficient": "a coefficient describing the scaling of u to be done by the simulator"
    }

In [6]:
labels= {
    "gsyn": "Single Synaptic Peak Conductance",
    "d": "Recovery from Depression Time Constant",
    "f": "Recovery from Facilitation Time Constant",
    "u": "Utilization of Synaptic Efficacy",
    "dtc": "Decay Time Constant",
    "nrrp": "Vesicles in Readily Releasable Pool",
    "gsynSRSF":  "Synaptic Receptor Scaling Factor",
    "uHillCoefficient": "uHill Coefficient"
    }

In [7]:
punits = {
    "gsyn": "nS",
    "d": "ms",
    "f": "ms",
    "u": "Unitless",
    "dtc": "ms",
    "nrrp": "Unitless",
    "gsynSRSF": "Unitless",
    "uHillCoefficient": "Unitless"
    }

In [8]:
combined_parameters = e1_yaml['parameters'] + probAMPA_yaml['parameters'] + probGABA_yaml['parameters']

## Read parameter ontology

In [9]:
base = '/Users/cgonzale/Documents/code/brain-modeling-ontology/ontologies/bbp'

In [10]:
parameter_ontology = rdflib.Graph()
parameter_ontology.parse(f"{base}/parameters.ttl")

<Graph identifier=N8be6d644734048f9a9e160156e54881e (<class 'rdflib.graph.Graph'>)>

In [11]:
def add_term(graph, new_s, label, units, description, pref_label=None, parent_label=None):
    triples_to_add = set()
    if parent_label:
        for s, p, o in graph.triples((None, RDFS.label, rdflib.term.Literal(parent_label, lang='en'))):
            triples_to_add.add((new_s, RDFS.subClassOf, s))
    triples_to_add.add((new_s, RDFS.label, rdflib.term.Literal(label, lang='en')))
    triples_to_add.add((new_s, RDF.type, OWL.Class))
    triples_to_add.add((new_s, SCHEMAORG.unitCode, rdflib.term.Literal(units)))
    triples_to_add.add((new_s, SKOS.definition, rdflib.term.Literal(description, lang='en')))
    if pref_label:
        triples_to_add.add((new_s, SKOS.prefLabel, rdflib.term.Literal(pref_label, lang='en')))
    for el in triples_to_add:
        if el not in graph:
            graph.add(el)

In [12]:
parent_label = "Model Brain Parameter"

In [13]:
done_parameters = []

## Add parameters from YAML file

In [14]:
def pretty_name(name):
    """Make a Class name pretty, capitalizing each word."""
    if '(' in name:
        name = name.replace('(', '')
        name = name.replace(')', '')
    if '/' in name:
        name = name.replace('/', '-')
    return re.sub(r"[A-Za-z]+('[A-Za-z]+)?", lambda mo: mo.group(0)[0].upper() + mo.group(0)[1:], name)

In [15]:
for parameter in combined_parameters:
    name = parameter['name']
    print(name)
    description = parameter['description']
    units = parameter['units']
    if len(name) < 4:
        if ',' in description:
            short_description = description.split(',')[0]
        else:
            short_description = description
        if '(' in short_description:
            re.sub(r'\([^)]*\)', '', short_description)
        idlabel = pretty_name(short_description).replace(' ', '')
        if '.' in idlabel:
            idlabel = idlabel.replace('.', '')
        iri = "https://bbp.epfl.ch/ontologies/core/parameters/"+idlabel
    else:
        iri = "https://bbp.epfl.ch/ontologies/core/parameters/"+name
    url = rdflib.URIRef(iri)
    if name not in done_parameters:
        add_term(parameter_ontology, url, name, units, description, parent_label=parent_label)
        done_parameters.append(name)

tau_r_NMDA
tau_d_NMDA
tau_r_AMPA
tau_d_AMPA
Use
Fac
Dep
Nrrp
tau_r_AMPA
tau_d_AMPA
tau_r_NMDA
tau_d_NMDA
Use
Dep
Fac
e
mg
slope_mg
scale_mg
gmax
u0
Nrrp
weight
NMDA_ratio
uHillCoefficient
tau_r_GABAA
tau_d_GABAA
tau_r_GABAB
tau_d_GABAB
Use
Dep
Fac
e_GABAA
e_GABAB
gmax
u0
Nrrp
weight
GABAB_ratio
uHillCoefficient


## Add parameters from xml

In [16]:
for parameter in labels:
    name = parameter
    print(name)
    description = definitions[parameter]
    unitcode = punits[parameter]
    if len(name) < 4:
        if ',' in description:
            short_description = description.split(',')[0]
        else:
            short_description = description
        if '(' in short_description:
            re.sub(r'\([^)]*\)', '', short_description)
        idlabel = pretty_name(short_description).replace(' ', '')
        if '.' in idlabel:
            idlabel = idlabel.replace('.', '')
        iri = "https://bbp.epfl.ch/ontologies/core/parameters/"+idlabel
    else:
        iri = "https://bbp.epfl.ch/ontologies/core/parameters/"+name
    url = rdflib.URIRef(iri)
    if name not in done_parameters:
        add_term(parameter_ontology, url, name, unitcode, description, pref_label=None, parent_label=parent_label)
        done_parameters.append(name)

gsyn
d
f
u
dtc
nrrp
gsynSRSF
uHillCoefficient


In [17]:
parameter_ontology.serialize(destination=f"{base}/parameters_updated.ttl")

<Graph identifier=N8be6d644734048f9a9e160156e54881e (<class 'rdflib.graph.Graph'>)>